## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [1]:
!pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
import pandas as pd

# Загрузка данных
recipes = pd.read_csv('recipes_sample.csv', index_col='id')
reviews = pd.read_csv('reviews_sample.csv', index_col='recipe_id')

# Приведение типов столбцов
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = pd.to_numeric(reviews['rating'])

recipes.info()
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 44123 to 298512
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            30000 non-null  object        
 1   minutes         30000 non-null  int64         
 2   contributor_id  30000 non-null  int64         
 3   submitted       30000 non-null  datetime64[ns]
 4   n_steps         18810 non-null  float64       
 5   description     29377 non-null  object        
 6   n_ingredients   21120 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 1.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 126696 entries, 57993 to 415599
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  126696 non-null  int64         
 1   user_id     126696 non-null  int64         
 2   date        126696 non-

In [7]:
import time

def calculate_mean_rating_iterrows(reviews):
    sum_rating = 0
    count = 0
    
    start_time = time.time()
    
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            sum_rating += row['rating']
            count += 1
    
    mean_rating = sum_rating / count if count != 0 else 0
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    return mean_rating, execution_time

mean_rating_iterrows, execution_time_iterrows = calculate_mean_rating_iterrows(reviews)
print("Mean rating (using iterrows):", mean_rating_iterrows)
print("Execution time (iterrows):", execution_time_iterrows)


Mean rating (using iterrows): 4.4544402182900615
Execution time (iterrows): 13.12532901763916


In [6]:
def calculate_mean_rating_iterrows_filtered(reviews):
    sum_rating = 0
    count = 0
    
    start_time = time.time()
    
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    for index, row in filtered_reviews.iterrows():
        sum_rating += row['rating']
        count += 1
    
    mean_rating = sum_rating / count if count != 0 else 0
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    return mean_rating, execution_time

mean_rating_iterrows_filtered, execution_time_iterrows_filtered = calculate_mean_rating_iterrows_filtered(reviews)
print("Mean rating (using iterrows and filtered):", mean_rating_iterrows_filtered)
print("Execution time (iterrows and filtered):", execution_time_iterrows_filtered)


Mean rating (using iterrows and filtered): 4.4544402182900615
Execution time (iterrows and filtered): 1.2215468883514404


In [8]:
def calculate_mean_rating_mean(reviews):
    start_time = time.time()
    
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    mean_rating = filtered_reviews['rating'].mean()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    return mean_rating, execution_time

mean_rating_mean, execution_time_mean = calculate_mean_rating_mean(reviews)
print("Mean rating (using mean):", mean_rating_mean)
print("Execution time (mean):", execution_time_mean)


Mean rating (using mean): 4.4544402182900615
Execution time (mean): 0.013514518737792969


In [10]:
mean_rating_mean == mean_rating_iterrows_filtered == mean_rating_iterrows

True

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [11]:
import line_profiler

# Создаем экземпляр профайлера
profiler = line_profiler.LineProfiler()

# Добавляем функции в профайлер
profiler.add_function(calculate_mean_rating_iterrows_filtered)
profiler.add_function(calculate_mean_rating_mean)

# Запускаем профилирование функций
profiler.enable_by_count()

# Вызываем функции, чтобы профайлер записал результаты
mean_rating_iterrows_filtered, execution_time_iterrows_filtered = calculate_mean_rating_iterrows_filtered(reviews)
mean_rating_mean, execution_time_mean = calculate_mean_rating_mean(reviews)

# Отключаем профайлер и выводим результаты
profiler.disable_by_count()
profiler.print_stats()


Timer unit: 1e-07 s

Total time: 4.43791 s
File: C:\Users\mehro\AppData\Local\Temp\ipykernel_1188\310949438.py
Function: calculate_mean_rating_iterrows_filtered at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calculate_mean_rating_iterrows_filtered(reviews):
     2         1         16.0     16.0      0.0      sum_rating = 0
     3         1          9.0      9.0      0.0      count = 0
     4                                               
     5         1         39.0     39.0      0.0      start_time = time.time()
     6                                               
     7         1     303063.0 303063.0      0.7      filtered_reviews = reviews[reviews['date'].dt.year == 2010]
     8     12094   36704037.0   3034.9     82.7      for index, row in filtered_reviews.iterrows():
     9     12094    7236267.0    598.3     16.3          sum_rating += row['rating']
    10     12094     135531.0     11.2      0.3

In [12]:
def calculate_mean_rating_apply(reviews):
    start_time = time.time()
    
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    mean_rating = filtered_reviews['rating'].apply(lambda x: pd.to_numeric(x)).mean()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    return mean_rating, execution_time

mean_rating_apply, execution_time_apply = calculate_mean_rating_apply(reviews)
print("Mean rating (using apply):", mean_rating_apply)
print("Execution time (apply):", execution_time_apply)


Mean rating (using apply): 4.4544402182900615
Execution time (apply): 0.07871294021606445


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [13]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [15]:
def get_word_reviews_count_optimized(df):
    word_reviews = {}
    word_reviews_count = {}

    def process_review(review):
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    df.dropna(subset=['review'], inplace=True)
    df['review'].apply(process_review)

    for word, reviews in word_reviews.items():
        word_reviews_count[word] = len(reviews)

    return word_reviews_count


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [16]:
import numpy as np
import numba
import timeit

# Версия без использования векторизованных операций и методов массивов numpy
def mape_vanilla(reviews):
    total_mape = 0.0
    count = 0
    for _, row in reviews.iterrows():
        rating = row['rating']
        recipe_id = row.get('recipe_id')
        if recipe_id is not None:
            average_rating = reviews[reviews['recipe_id'] == recipe_id]['rating'].mean()
            if not np.isnan(rating) and not np.isnan(average_rating):
                total_mape += abs((rating - average_rating) / average_rating)
                count += 1
    return total_mape / count if count > 0 else 0.0

# Версия с использованием numba, но без векторизации numpy
@numba.jit
def mape_numba(reviews):
    total_mape = 0.0
    count = 0
    for _, row in reviews.iterrows():
        rating = row['rating']
        recipe_id = row.get('recipe_id')
        if recipe_id is not None:
            average_rating = reviews[reviews['recipe_id'] == recipe_id]['rating'].mean()
            if not np.isnan(rating) and not np.isnan(average_rating):
                total_mape += abs((rating - average_rating) / average_rating)
                count += 1
    return total_mape / count if count > 0 else 0.0

# Версия с использованием векторизованных операций и методов массивов numpy
def mape_numpy(reviews):
    ratings = reviews['rating']
    recipe_ids = reviews['recipe_id']
    unique_ids = np.unique(recipe_ids)
    average_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    mask = ~np.isnan(ratings) & ~np.isnan(average_ratings)
    total_mape = np.mean(np.abs((ratings - average_ratings) / average_ratings)[mask])
    return total_mape

# Версия с использованием векторизованных операций и методов массивов numpy и numba
@numba.jit
def mape_numpy_numba(reviews):
    ratings = reviews['rating']
    recipe_ids = reviews['recipe_id']
    unique_ids = np.unique(recipe_ids)
    average_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    mask = ~np.isnan(ratings) & ~np.isnan(average_ratings)
    total_mape = np.mean(np.abs((ratings - average_ratings) / average_ratings)[mask])
    return total_mape

# Измерение времени выполнения функции mape_vanilla
time_vanilla = timeit.timeit(lambda: mape_vanilla(reviews), number=1)

# Измерение времени выполнения функции mape_numba
time_numba = timeit.timeit(lambda: mape_numba(reviews), number=1)

# Измерение времени выполнения функции mape_numpy
time_numpy = timeit.timeit(lambda: mape_numpy(reviews), number=1)

# Измерение времени выполнения функции mape_numpy_numba
time_numpy_numba = timeit.timeit(lambda: mape_numpy_numba(reviews), number=1)

# Вывод результатов
print("Время выполнения функции mape_vanilla:", time_vanilla)
print("Время выполнения функции mape_numba:", time_numba)
print("Время выполнения функции mape_numpy:", time_numpy)
print("Время выполнения функции mape_numpy_numba:", time_numpy_numba)



C:\Users\mehro\AppData\Local\Temp\ipykernel_9004\2178603785.py:20: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "mape_numba" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\mehro\AppData\Local\Temp\ipykernel_9004\2178603785.py (22)

File "..\..\..\AppData\Local\Temp\ipykernel_9004\2178603785.py", line 22:
<source missing, REPL/exec in use?>

  @numba.jit
C:\Users\mehro\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "mape_numba" was compiled in object mode without forceobj=True.

File "..\..\..\AppData\Local\Temp\ipykernel_9004\2178603785.py", line 20:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\mehro\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is 

KeyError: 'recipe_id'

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4